# **前日の20:00までのデータを用いて翌日の00:00~23:30までの日射時間(sl)をMLPで予測する**

### **必要な関数・ライブラリ**

In [1]:
# データ加工・処理・分析モジュール
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import random

%matplotlib inline

import time

In [2]:
def set_time(dataframe, col_name):
    '''
    to_datetimeを使うための前処理
    '''
    dataframe[col_name] = dataframe[col_name].map(lambda x : transform_time(x))
    return dataframe

In [3]:
def transform_time(x):
    '''
    set_time内で使う関数
    to_datetimeで24時をサポートしないので00に変更する処理
    '''
    str_x = str(x)
    res = ''
    if str(x)[8:10] == '24':
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' 00:'+str_x[10:12] 
    else:
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' '+ str_x[8:10] +':'+str_x[10:12]
    return res

In [4]:
def normalize_array(x):
    '''
    min, max, min-max正規化を行なった配列(np.array)を返す
    '''
    x = np.array(x)
    x_min = x.min()
    x_max = x.max()
    normalized = (x - x_min) / (x_max - x_min) 
    return x_min, x_max, normalized

In [5]:
def denormalize_array(normalized_x, x_min, x_max):
    '''
    正規化前のmin, maxを用いて元のスケールに戻す
    '''
    normalized_x = np.array(normalized_x)
    denormalize_array = (normalized_x) * (x_max - x_min) + x_min
    return denormalize_array

In [40]:
def get_input_data(X):
    '''
    全時系列データから00:00 ~ 20:00までのデータごとに分割する
    '''
    input_list = []
    # 一日は48個の時間帯に分けられる
    total_size = len(X) // 48
    reshaped_size = X[0:41].shape[0] * X[0:41].shape[1]
    for i in range(total_size+1):
        each_data = X[(i*48):(i*48+41)]
        reshaped = each_data.reshape(reshaped_size)
        input_list.append(list(reshaped))
    input_list = np.array(input_list)
    return input_list

In [39]:
def get_output_data(Y):
    '''
    全時系列データから00:00 ~ 23:30までのデータごとに分割する
    '''
    output_list = []
    # 一日は48個の時間帯に分けられる
    total_size = len(Y) // 48
    for i in range(total_size):
        each_data = Y[(i*48):(i*48+48)]
        reshaped = each_data.reshape(48)
        output_list.append(list(reshaped))
    output_list = np.array(output_list)
    return output_list

In [8]:
def drop_nan(X, Y):
    '''
    正解データがnanであるデータの組を削除
    '''
    mask = np.isnan(Y)
    X = X[~mask]
    Y = Y[~mask]
    return X, Y

In [9]:
def calc_mae(X, Y):
    '''
    X, Yがメモリを食いすぎるのでfor文で計算
    '''
    mse = 0
    for i in range(len(X)):
        mse += np.abs(X[i]- Y[i])
    return mse/len(X)

### **データの準備**

In [10]:
# 予測する発電所番号を決める
target_place = 1
chunk_size = 12
model_name = "model_"+str(target_place)+'_chunk_'+str(chunk_size)
try:
    os.mkdir('./data/'+model_name)
except:
    print("file exists")

file exists


In [21]:
# アメダスデータの読み込み
if target_place == 1 or target_place == 2:
    # 名古屋アメダスのデータを使って予測する, amd_51106
    # 対象アメダスは横浜アメダス, amd_46106
    # 各amdidはamd_masterに記載されている
    amd_data = pd.read_csv('data/raw_data/amd_51106.tsv', delimiter = '\t')
    amd_data = set_time(amd_data, 'datetime')
    amd_data['datetime'] = amd_data['datetime'].map(lambda x : pd.to_datetime(x))
    target_amd_data = pd.read_csv('data/raw_data/amd_46106.tsv', delimiter = '\t')
    target_amd_data = set_time(target_amd_data, 'datetime')
    target_amd_data['datetime'] = target_amd_data['datetime'].map(lambda x : pd.to_datetime(x))
elif target_place == 3:
    # 甲府アメダスのデータを使って予測する, amd_49142
    # 各amdidはamd_masterに記載されている
    amd_data = pd.read_csv('data/raw_data/amd_49142.tsv', delimiter = '\t')
    amd_data = set_time(amd_data, 'datetime')
    amd_data['datetime'] = amd_data['datetime'].map(lambda x : pd.to_datetime(x))
else:
    raise ValueError("invalid input target_place_num")

In [24]:
# 30分毎のデータに編集
amd_data = amd_data[["datetime", "pr", "max_ws", "ave_ws", "max_tp", "min_tp", "sl"]]

# 欠損値を一つ前の値で置換/output_data
amd_data = amd_data.fillna(method='bfill')
amd_30 = amd_data.set_index('datetime').groupby(pd.TimeGrouper(freq='1800s', closed='left')).sum()

# datetimeのカラムを復活させる
amd_30['datetime'] = amd_30.index
amd_30.index = np.arange(len(amd_30))
amd_30.head()

,pr,max_ws,ave_ws,max_tp,min_tp,sl,datetime
0,0,68.0,41.0,73.0,68.0,0.0,2012-01-01 00:00:00
1,0,81.0,56.0,46.0,39.0,0.0,2012-01-01 00:30:00
2,0,81.0,52.0,43.0,40.0,0.0,2012-01-01 01:00:00
3,0,67.0,40.0,44.0,39.0,0.0,2012-01-01 01:30:00
4,0,33.0,20.0,43.0,37.0,0.0,2012-01-01 02:00:00


In [27]:
# 30分毎のデータに編集
target_amd_data = target_amd_data[["datetime", "sl"]]

# 欠損値を一つ前の値で置換/output_data
target_amd_data = target_amd_data.fillna(method='bfill')
target_30 = target_amd_data.set_index('datetime').groupby(pd.TimeGrouper(freq='1800s', closed='left')).sum()

# datetimeのカラムを復活させる
target_30['datetime'] = target_30.index
target_30.index = np.arange(len(target_30))
target_30.head()

,sl,datetime
0,0.0,2012-01-01 00:00:00
1,0.0,2012-01-01 00:30:00
2,0.0,2012-01-01 01:00:00
3,0.0,2012-01-01 01:30:00
4,0.0,2012-01-01 02:00:00


In [28]:
# モデル構築のためにデータを分割する

# 学習に必要なデータ
# 前日の00:00 ~ 20:00のデータで翌日の0:00 ~ 23:30を予測する
train_x_s_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2012-01-01 00:00')].index[0]
train_x_e_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2015-12-30 20:00')].index[0]
train_y_s_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2012-01-02 00:00')].index[0]
train_y_e_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2015-12-31 23:30')].index[0]

output_tp = target_30["sl"][train_y_s_idx:train_y_e_idx+1]
input_data = amd_30[[ "pr", "max_ws", "ave_ws", "max_tp", "min_tp", "sl"]][train_x_s_idx:train_x_e_idx+1]

# 予測に必要なデータ
# 前日の00:00 ~ 20:00のデータで翌日の0:00 ~ 23:30を予測する
# test_y_s_idx = amd_data[amd_data['datetime'] == pd.to_datetime('2015-12-31 00:00')].index[0]
# test_y_e_idx = amd_data[amd_data['datetime'] == pd.to_datetime('2017-3-31 23:50')].index[0]
# test_x_s_idx = test_y_s_idx - chunk_size
# test_x_e_idx = amd_data[amd_data['datetime'] == pd.to_datetime('2017-3-31 23:40')].index[0]

# test_input_tp = amd_data[test_x_s_idx:test_x_e_idx+1]
# test_output_tp = amd_data[test_y_s_idx:test_y_e_idx+1]

In [46]:
# MLPに突っ込むための準備をする

# rnnに突っ込むためにmin-max正規化しておく
normalized_input = (input_data - input_data.min()) / (input_data.max() - input_data.min())
normalized_input = np.array(normalized_input)
normalized_output = (output_tp - output_tp.min()) / (output_tp.max() - output_tp.min())
normalized_output = np.array(normalized_output)

input_list = get_input_data(normalized_input)
output_list = get_output_data(normalized_output)

# testデータの入力を用意
# normalized_test_input = (test_input_tp - test_input_tp.min()) / (test_input_tp.max() - test_input_tp.min())
# normalized_test_input = np.array(normalized_test_input)
# test_input_list = get_chunked_data(normalized_test_input, chunk_size)

# denormalize用
output_max = float(output_tp.max())
output_min = float(output_tp.min())

# RNNに突っ込むためにデータを整形
X = np.array(input_list).reshape(input_list.shape[0], input_list.shape[1], 1)
Y = np.array(normalized_output).reshape(output_list.shape[0], output_list.shape[1], 1)
# X_predict = np.array(test_input_list).reshape(len(test_input_list), chunk_size, test_input_list.shape[2])

### **モデルの構築**

条件:翌日の気温を予測するには前日の20:00までの気象情報しか使えない

モデル:翌日の30分毎の気温を前日の20:00までの各所のアメダスの気象情報で予測させる

入力:各所のアメダス情報の配列

出力:0:00, 0:30, .... , 23:00, 23:30の気温の配列

とりあえず一つの箇所のデータを使って予測
